In [88]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegressionCV, LinearRegression, LogisticRegression
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.model_selection import TimeSeriesSplit
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingRandomSearchCV
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, mean_squared_error, roc_auc_score
import numpy as np
import joblib

In [89]:
data = pd.read_parquet('./input/creditos_hist.parquet') # Base de la Central de Deudores histórica

In [90]:
data = data.sort_values(by=["identificacion", "entidad", "periodo"])

data["buen_historial"] = data.groupby(["identificacion", "entidad"])["situacion"].expanding().min().reset_index(level=[0,1], drop=True).eq(0).astype(int)

# La literatura indica que también importa la duración de la relación empresa-banco, por lo que contamos la cantidad 
# de periodos que aparece cada par: empresa-banco
data["n_periodos"] = data.groupby(["identificacion", "entidad"]).cumcount()

In [91]:
# Eliminamos las situaciones 0, que indican que el crédito ya fue pagado
data = data.loc[data['situacion'] != 0]
data = data.drop('denominacion', axis = 1) # Elimino la columna con las razones sociales para ahorrar RAM

In [92]:
# También puede importar la cantidad de meses en los cuales una relación entre banco y empresa se encuentra "activa"
data['periodo_int'] = data['periodo'].astype('int')

# Calcular la diferencia entre el periodo actual y el anterior dentro de cada grupo
data["diff"] = data.groupby(["identificacion", "entidad"])["periodo_int"].diff()

# Identificar nuevas secuencias (cuando hay un salto en los periodos)
data["nueva_secuencia"] = (data["diff"] != 1) & data["diff"].notna()

# Contar la cantidad de periodos consecutivos en los que la relación estuvo activa
data["n_periodos_activo"] = data.groupby(["identificacion", "entidad"])["nueva_secuencia"].cumsum()

# Eliminar la columna auxiliar
data.drop(columns=["diff", "nueva_secuencia", 'periodo_int'], inplace=True)

In [93]:
data = data.sort_values(by=['identificacion', 'periodo'], ascending= False) # Ordenamos de acuerdo a cada empresa y periodo

In [94]:
# Una variable que puede ser de interés es cuantos créditos tiene una empresa en un momento dado del tiempo
counts = data.groupby(['identificacion', 'periodo']).size().reset_index(name='n_creditos')

# También nos interesa cuanta plata debe una empresa en cada momento dado
sums = data.groupby(['identificacion', 'periodo'], as_index=True)['monto'].sum().reset_index(name='sum_montos')

# Definimos como default cuando el crédito se encuentra en situación 4 o 5, por lo que creamos la dummy de default
# Esta es nuestra variable dependiente
data['default'] = (data['situacion'] >= 4).astype(int)

In [95]:
# Queremos predecir el default el periodo siguiente
data['default_lag'] = data.groupby(['identificacion', 'entidad'])['default'].shift(1) # Lag a la variable default
data = data.dropna(subset=['default_lag']) # Eliminamos las observaciones que no tienen variable dependiente
data['default_lag'] = data['default_lag'].astype(int) # Cambio el dtype de la variable de interés

In [96]:
data["sin_historial"] = (
    data.groupby("identificacion")["periodo"]
    .transform("rank", method="first") == 1).astype(int)

# Puede influir el hecho de que la empresa no tenga historial crediticio
# Creamos una variable que indica si es la primera vez que aparece en la base

In [97]:
data["prev_default_general"] = (
    data.groupby("identificacion")["default"].expanding().max().reset_index(level=0, drop=True).astype(int)
)

data["prev_default_entidad"] = (
    data.groupby(["identificacion", 'entidad'])["default"].expanding().max().reset_index(level=[0,1], drop=True).astype(int)
)

In [98]:
default_max = (
    data.groupby(["identificacion", "entidad"])["default"]
    .expanding()
    .max()
    .reset_index(level=[0, 1], drop=True)
)

data["buen_historial"] = ((data['buen_historial'] == 1) & (default_max == 0)).astype(int)

del default_max

In [99]:
# Agregamos las nuevas variables al dataframe
data = data.merge(counts, on=['identificacion', 'periodo'], how='left')
data = data.merge(sums, on=['identificacion', 'periodo'], how='left')

del sums, counts # Para ahorrar RAM

In [100]:
# Por último, la literatura también resalta que la intensidad de la relación empresa-banco es relevante
# Usamos como proxy para la intensidad la proporción del monto adeudado con un banco sobre el total adeudado
data['monto_relativo'] = data['monto'] / data['sum_montos']

In [101]:
# Convertir "periodo" a formato de fecha
data["periodo_fecha"] = pd.to_datetime(data["periodo"].astype(str), format="%Y%m")

# Calcular la cantidad de meses transcurridos desde el primer periodo
min_periodo = data["periodo_fecha"].min()
data["desde_inicio"] = ((data["periodo_fecha"].dt.year - min_periodo.year) * 12 + 
                              (data["periodo_fecha"].dt.month - min_periodo.month))

In [102]:
data = data.loc[data['periodo'] > '202310'] # Entrenamos el modelo solo con datos a partir de octubre de 2023

In [103]:
# Elijo aleatoriamente un porcentaje de las empresas de la población
np.random.seed(42)
cuits = data['identificacion'].unique()
moneda = np.random.binomial(1, 0.2, len(cuits)) # Es como tirar una moneda sesgada para que agarre un porcentaje arbitrario de las empresas 
cuits_aleatorios = cuits[moneda == 1] # Estos son los cuits con los que me voy a quedar
data = data.loc[data['identificacion'].isin(cuits_aleatorios)] # Me quedo unicamente con las obs que tienen un cuit dentro de los seleccionados aleatoriamente

del cuits, moneda, cuits_aleatorios

In [104]:
pv = pd.read_parquet('./input/principales_variables.parquet') # Datos de principales variables monetarias provenientes de la API del BCRA
pv.reset_index(inplace= True) # el index es la fecha, así que lo paso a columna
pv['fecha'] = pd.to_datetime(pv['fecha']) # paso la nueva columna al formato correcto
pv['periodo'] = pv['fecha'].dt.strftime('%Y%m') # armo una variable llamada periodo igual a la que tengo en los datos de la Central de Deudores
pv = pv.drop('fecha', axis = 1).groupby('periodo').agg(['mean', 'std']) # elimino la de "fecha" porque no me interesan los datos diarios
# Me quedo únicamente con los promedios por mes y también calculo el desvío estándar
pv.columns = ['_'.join(col).strip() for col in pv.columns] # Renombro las columnas para que sea más prolijo
pv.reset_index(inplace= True) # Vuelvo a agregar la columna periodo
pv = pv.loc[pv['periodo'].astype(int) <= 202412] # En la Central de Deudores tenemos datos hasta 202412
pv = pv.dropna(axis = 1) # Elimino las columnas con NAs

In [105]:
pv = pv.sort_values("periodo")

pv["inflacion_acumulada"] = (1 + pv["Inflación mensual (variación en %)_mean"]/100).cumprod() # Calculo la inflación acumulada

columnas = pv.columns.tolist()
tasas = [3, 4, 5, 6, 28, 29, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53] # Estos son los índices de las columnas que tienen tasas
excluir = [0, 11, 12, 25, 26, 27, 34, 35, 58, 59, 60, 61, 62, 63, 64] # Estos son los índices de las columnas que ya están en valores reales

columnas_excluir = [columnas[i] for i in excluir]
columnas_tasas = [columnas[i] for i in tasas]

columnas_nominales = [col for col in pv.columns if col not in columnas_excluir and col not in columnas_tasas] # El resto son columnas en valores nominales

for col in columnas_nominales:
    nombre_col_real = f"{col}_real"
    pv[nombre_col_real] = pv[col] / pv['inflacion_acumulada'] # Pasamos los valores de las columnas nominales a valores reales del primer periodo

for col in columnas_tasas:
    nombre_col_real = f"{col}_real"
    pv[col] = pv[col]/100
    pv[nombre_col_real] = pv[col]/pv['inflacion_acumulada'] # A las tasas también las pasamos a valores reales, pero antes las dividimos pro 100
    
    
del col, columnas, columnas_excluir, columnas_tasas, columnas_nominales, tasas, excluir, nombre_col_real

In [106]:
data = data.merge(pv, on = 'periodo', how = 'left') # Junto las principales variables monetarias con la Central de Deudores

In [107]:
data['monto_real'] = data['monto']/data['inflacion_acumulada'] # Pasamos el monto del outstanding a valor real
data['sum_montos_real'] = data['sum_montos']/data['inflacion_acumulada'] # Lo mismo con el monto total adeudado
data.drop('inflacion_acumulada', axis = 1, inplace = True)

In [108]:
emae = pd.read_excel('./input/sh_emae_mensual_base2004.xls', index_col=[0,1]) # EMAE mensual
meses_a_numeros = {
    'Enero': '01', 'Febrero': '02', 'Marzo': '03', 'Abril': '04',
    'Mayo': '05', 'Junio': '06', 'Julio': '07', 'Agosto': '08',
    'Septiembre': '09', 'Octubre': '10', 'Noviembre': '11', 'Diciembre': '12'
} # Diccionario para pasar los meses a números
emae = emae.reset_index()
emae['level_1'] = emae['level_1'].map(meses_a_numeros) # Pasamos los meses a números
emae['periodo'] = emae['Período'].astype(str) + emae['level_1'] # Armamos la variable periodo
emae = emae.drop(columns=['level_1', 'Período'])

del meses_a_numeros

In [109]:
data = data.merge(emae, on = 'periodo', how= 'left') # Juntamos las bases

In [110]:
arca = pd.read_parquet('./input/constancia_inscripcion.parquet') # Cargo los datos de la constancia de inscripción de ARCA

In [111]:
cuits_arca = set(arca['identificacion']) # Los cuits que están en la base de ARCA
cuits_bcra = set(data['identificacion']) # Los cuits que están en la Central de Deudores

faltan = list(cuits_bcra - cuits_arca) # Si están en la Central de Deudores pero no en ARCA es por alguna irregularidad en ARCA

# Creemos que tener irregularidades en ARCA puede ser buen predictor de default
data['sin_arca'] = (data['identificacion'].isin(faltan)).astype(int)  # Creamos la variable

del cuits_arca, cuits_bcra, faltan

In [112]:
# Ponemos bien el tipo de dato para las columnas categóricas, así el get_dummies funciona bien
data['identificacion'] = data['identificacion'].astype('category')
data['entidad'] = data['entidad'].astype('category')
data['situacion'] = data['situacion'].astype('category')
data['default'] = data['default'].astype('category')
data['periodo'] = data['periodo'].astype('category')
data['default_lag'] = data['default_lag'].astype('category')
data['prev_default_entidad'] = data['prev_default_entidad'].astype('category')
data['prev_default_general'] = data['prev_default_general'].astype('category')
data['sin_arca'] = data['sin_arca'].astype('category')
data['buen_historial'] = data['buen_historial'].astype('category')
data['sin_historial'] = data['sin_historial'].astype('category')

In [113]:
data.columns = data.columns.str.replace('\n', ' ') # Elimino los saltos de línea

In [114]:
data.to_parquet('./input/data_limpio_02.parquet')
#data = pd.read_parquet('./input/data_limpio.parquet')

Para hacer cross validation, tenemos que tener en cuenta que tenemos un panel. Por lo tanto, vamos a entrenar el modelo con datos del pasado y evaluarlo con datos del futuro

In [115]:
# Cross Validation
data.sort_values(by=['periodo'], ascending= True, inplace = True) # Ordeno de acuerdo a la fecha
data = data.reset_index().drop(columns= 'index')
split_index = int(len(data) * 0.8) # El 80% de las observaciones más antiguas
train_indices = data.iloc[:split_index].index # Estos son los índices con los que después voy a separar en test y train
test_indices = data.iloc[split_index:].index

In [116]:
# Variable dependiente
Y = data['default_lag']

In [117]:
columnas = ['desde_inicio', 'monto', 'n_creditos', 'sum_montos', 'n_periodos', 'n_periodos_activo', 'monto_relativo', 'sin_arca', 'default', 'prev_default_entidad', 'prev_default_general', 'sin_historial', 'monto_real', 'sum_montos_real', 'buen_historial'] # Algunas de las variables independientes del modelo
# Si agrego efectos fijos por tiempo, "periodo" va después de "sin_historial"
pv.set_index('periodo', inplace= True)
pv.drop('inflacion_acumulada', axis = 1, inplace = True)
columnas.extend(pv.columns) # Todas las columnas de las principales variables monetarias
emae.set_index('periodo', inplace= True)
columnas.extend(emae.columns) # Todas las columnas de emae

In [118]:
with open('./input/columnas.txt', 'w', encoding = 'utf-8') as f:
    for col in columnas:
        clean_col = col.replace('\n', ' ')
        f.write(f'{clean_col}\n')
        
del col, clean_col, f

In [119]:
data.columns = [col.replace('\n', ' ') for col in data.columns]
columnas = [col.replace('\n', ' ') for col in columnas]
X = pd.get_dummies(data[columnas], drop_first=True) # Meto las columnas en get_dummies

In [120]:
X_cambio_index = X.loc[X['default_1'] != Y].index # Estas son las observaciones que cambiaron de estado entre t y t+1
#X = X.drop('default_1', axis = 1) # Si no queremos que sea autorregresivo droppeamos la variable laggeada

# Separo en entrenamiento y test
X_train = X.loc[train_indices]
Y_train = Y.loc[train_indices]
X_test = X.loc[test_indices]
Y_test = Y.loc[test_indices]

# También identificamos las observaciones en el conjunto de entrenamiento que cambiaron de estado
X_test_cambio_index =list(set(X_test.index) & set(X_cambio_index))
Y_test_cambio_index = list(set(Y_test.index) & set(X_cambio_index))

X_test_cambio = X_test.loc[X_test_cambio_index]
Y_test_cambio = Y_test.loc[Y_test_cambio_index]

del train_indices, test_indices, X_test_cambio_index, Y_test_cambio_index, split_index, X_cambio_index

In [121]:
# Función para evaluar los modelos
def eval(model, X_test, Y_test, linear = None):
    y_pred = model.predict(X_test)
    
    if linear:
        y_pred = np.where(y_pred >= 0.5, 1, 0)
    
    cm = confusion_matrix(Y_test, y_pred)
    
    #precision = precision_score(Y_test, y_pred)
    #recall = recall_score(Y_test, y_pred)
    f1 = f1_score(Y_test, y_pred)
    accuracy = accuracy_score(Y_test, y_pred)
    mse = mean_squared_error(Y_test, y_pred)
    auc = roc_auc_score(Y_test, y_pred)
    
    print(cm)
    #print(f'La precisión es: {precision}')
    #print(f'El recall es: {recall}')
    print(f'El f1 es: {f1}')
    print(f'El accuracy es: {accuracy}')
    print(f'El MSE es: {mse}')
    print(f'El AUC es: {auc}')
    
    return y_pred 

In [122]:
# Función para evaluar el MSE promedio y su desvío estándar en los distintos folds para cada lambda
def mse_table(model, elasticnet=None):
    inverse_Cs = 1 / model.Cs_
    results = []
    
    if elasticnet:
        l1_ratios = model.l1_ratios_
        for c_idx, lambda_ in enumerate(inverse_Cs):
            for l1_idx, l1_ratio in enumerate(l1_ratios):
                    mean_score = np.mean(-model.scores_[1][:, c_idx, l1_idx], axis=0)
                    std = np.std(-model.scores_[1][:, c_idx, l1_idx], axis=0)
                    results.append({
                        "Lambda": lambda_,
                        "L1 Ratio": l1_ratio,
                        "Mean MSE": mean_score,
                        "Std MSE": std
                    })
        
        results_table = pd.DataFrame(results).sort_values(by="Mean MSE", ascending=True)
    
    else:
        mean_scores = np.mean(-model.scores_[1], axis=0)
        std = np.std(-model.scores_[1], axis=0)
        results_table = pd.DataFrame({
            "Lambda": inverse_Cs,
            "Mean Score": mean_scores,
            "Std MSE": std
        }).sort_values(by="Mean Score", ascending=True)
    
    print(results_table)
    return results_table

In [123]:
# Función para quedarmos con los coeficientes que son distintos de 0
def non_zero_coefs(model, X_train):
    best_coefs = model.coef_[0]
    feature_names = X_train.columns
    non_zero_coefs = []
    for coef, name in zip(best_coefs, feature_names):
        if coef != 0:
            non_zero_coefs.append({
                "Variable": name,
                "Coeficiente": coef,
                'coef_abs': abs(coef)
            })
    non_zero_table = pd.DataFrame(non_zero_coefs).sort_values(by = 'coef_abs', ascending= False)
    print(non_zero_table.iloc[: , :2])
    
    return non_zero_table

## LASSO

In [124]:
start = -4
end = 4

tscv = TimeSeriesSplit(n_splits=10) # Este cross validation tiene en cuenta la temporarlidad de la base

pipeline = Pipeline([
    ('scaler', StandardScaler()), # Primero estandariza los datos
    ('logreg', LogisticRegressionCV( # Estima el modelo usando cross validation para elegir el mejor hiperparámetro
        cv=tscv,
        penalty='l1',
        solver='saga',
        scoring='neg_mean_squared_error',
        max_iter=2000,
        random_state= 42,
        tol = 1e-3,
        n_jobs= -1,
        fit_intercept= True,
        Cs= np.logspace(start, end, 20)
    ))
])

del start, end

In [125]:
pipeline.fit(X_train, Y_train) # Entreno el modelo

Pipeline(steps=[('scaler', StandardScaler()),
                ('logreg',
                 LogisticRegressionCV(Cs=array([1.00000000e-04, 2.63665090e-04, 6.95192796e-04, 1.83298071e-03,
       4.83293024e-03, 1.27427499e-02, 3.35981829e-02, 8.85866790e-02,
       2.33572147e-01, 6.15848211e-01, 1.62377674e+00, 4.28133240e+00,
       1.12883789e+01, 2.97635144e+01, 7.84759970e+01, 2.06913808e+02,
       5.45559478e+02, 1.43844989e+03, 3.79269019e+03, 1.00000000e+04]),
                                      cv=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=10, test_size=None),
                                      max_iter=2000, n_jobs=-1, penalty='l1',
                                      random_state=42,
                                      scoring='neg_mean_squared_error',
                                      solver='saga', tol=0.001))])

In [126]:
joblib.dump(pipeline, './output/lasso_02_autorreg.pkl')

['./output/lasso_02_autorreg.pkl']

In [127]:
y_pred = eval(pipeline, X_test, Y_test)

[[176108     60]
 [   593  11076]]
El f1 es: 0.9713659285244464
El accuracy es: 0.9965235816159755
El MSE es: 0.0034764183840244466
El AUC es: 0.9744205041324834


In [128]:
lasso = pipeline.named_steps['logreg'] # Agarro el modelo desde el pipeline
best_c_lasso = lasso.C_[0]
print(f'El mejor lambda para el modelo es: {1/best_c_lasso}')

El mejor lambda para el modelo es: 3792.6901907322494


In [129]:
mse = mse_table(lasso)

          Lambda  Mean Score   Std MSE
1    3792.690191    0.016731  0.002600
2    1438.449888    0.016731  0.002600
3     545.559478    0.016731  0.002600
4     206.913808    0.016731  0.002600
5      78.475997    0.016731  0.002600
6      29.763514    0.016731  0.002600
7      11.288379    0.018593  0.004267
9       1.623777    0.018740  0.004218
8       4.281332    0.018740  0.004218
10      0.615848    0.018769  0.004255
11      0.233572    0.019109  0.004786
12      0.088587    0.019513  0.004994
13      0.033598    0.019811  0.005236
14      0.012743    0.020180  0.005572
15      0.004833    0.020297  0.005691
16      0.001833    0.020461  0.005823
17      0.000695    0.020543  0.005857
18      0.000264    0.020672  0.006025
19      0.000100    0.020778  0.006116
0   10000.000000    0.042656  0.077903


In [184]:
non_zero_lasso = non_zero_coefs(lasso, X_train)

                 Variable  Coeficiente
0               default_1     1.227805
1  prev_default_entidad_1     1.221713


In [131]:
y_pred = eval(pipeline, X_test_cambio, Y_test_cambio)

[[  0  60]
 [593   0]]
El f1 es: 0.0
El accuracy es: 0.0
El MSE es: 1.0
El AUC es: 0.0


In [132]:
y_pred = eval(pipeline, X, Y)

[[876843    768]
 [  3034  58538]]
El f1 es: 0.9685467992521385
El accuracy es: 0.995951800660787
El MSE es: 0.004048199339212912
El AUC es: 0.9749246261239876


## RandomForest

In [133]:
rf = RandomForestClassifier()

tscv = TimeSeriesSplit(n_splits = 10)

param_dist = {
    'n_estimators': list(range(100, 701, 10)),  
    'max_depth': [None] + list(range(10, 71, 5)), 
}

halving_search = HalvingRandomSearchCV(
    estimator=rf,
    param_distributions=param_dist,
    factor=3,  
    min_resources=10000,  
    resource="n_samples",  
    cv=tscv,
    scoring='neg_mean_squared_error',
    n_jobs=-1,
    random_state=42,
    verbose=4
)

In [134]:
halving_search.fit(X_train, Y_train)

n_iterations: 4
n_required_iterations: 4
n_possible_iterations: 4
min_resources_: 10000
max_resources_: 751346
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 75
n_resources: 10000
Fitting 10 folds for each of 75 candidates, totalling 750 fits
----------
iter: 1
n_candidates: 25
n_resources: 30000
Fitting 10 folds for each of 25 candidates, totalling 250 fits
----------
iter: 2
n_candidates: 9
n_resources: 90000
Fitting 10 folds for each of 9 candidates, totalling 90 fits
----------
iter: 3
n_candidates: 3
n_resources: 270000
Fitting 10 folds for each of 3 candidates, totalling 30 fits


HalvingRandomSearchCV(cv=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=10, test_size=None),
                      estimator=RandomForestClassifier(), min_resources=10000,
                      n_jobs=-1,
                      param_distributions={'max_depth': [None, 10, 15, 20, 25,
                                                         30, 35, 40, 45, 50, 55,
                                                         60, 65, 70],
                                           'n_estimators': [100, 110, 120, 130,
                                                            140, 150, 160, 170,
                                                            180, 190, 200, 210,
                                                            220, 230, 240, 250,
                                                            260, 270, 280, 290,
                                                            300, 310, 320, 330,
                                                            340, 350, 360, 370,
                                                            380, 390, ...]},
                      random_state=42, scoring='neg_mean_squared_error',
                      verbose=4)

In [135]:
y_pred = eval(halving_search, X_test, Y_test)

[[176032    136]
 [   434  11235]]
El f1 es: 0.9752604166666666
El accuracy es: 0.9969654540905146
El MSE es: 0.0030345459094853517
El AUC es: 0.9810177240697358


In [136]:
joblib.dump(halving_search, './output/rf_02_autorreg.pkl')

['./output/rf_02_autorreg.pkl']

In [137]:
results_rf = pd.DataFrame(halving_search.cv_results_)
results_rf.to_csv('./output/results_rf_02_autorreg.csv')

In [138]:
results_rf.sort_values(by = 'mean_test_score')
results_rf.head(10)

,iter,n_resources,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_n_estimators,param_max_depth,params,split0_test_score,...,split2_train_score,split3_train_score,split4_train_score,split5_train_score,split6_train_score,split7_train_score,split8_train_score,split9_train_score,mean_train_score,std_train_score
0,0,10000,1.661910,0.752246,0.038385,0.009535,150,10,"{'n_estimators': 150, 'max_depth': 10}",-0.0022,...,-0.0,-0.000550,-0.00066,-0.0,-0.0,-0.000138,-0.001222,-0.00088,-0.000345,0.000428
1,0,10000,2.072373,1.091685,0.044575,0.007309,170,40,"{'n_estimators': 170, 'max_depth': 40}",-0.0022,...,-0.0,-0.000275,-0.00022,-0.0,-0.0,-0.000000,-0.000244,-0.00011,-0.000085,0.000111
2,0,10000,3.108040,1.342532,0.061028,0.003307,250,20,"{'n_estimators': 250, 'max_depth': 20}",-0.0022,...,-0.0,-0.000275,-0.00022,-0.0,-0.0,-0.000000,-0.000244,-0.00011,-0.000085,0.000111
3,0,10000,4.341509,1.819803,0.085608,0.007544,390,20,"{'n_estimators': 390, 'max_depth': 20}",-0.0022,...,-0.0,-0.000275,-0.00022,-0.0,-0.0,-0.000000,-0.000244,-0.00011,-0.000085,0.000111
4,0,10000,7.383132,3.421192,0.144135,0.007306,700,65,"{'n_estimators': 700, 'max_depth': 65}",-0.0022,...,-0.0,-0.000275,-0.00022,-0.0,-0.0,-0.000000,-0.000244,-0.00011,-0.000085,0.000111
5,0,10000,7.294696,3.540629,0.144754,0.010090,650,45,"{'n_estimators': 650, 'max_depth': 45}",-0.0033,...,-0.0,-0.000275,-0.00022,-0.0,-0.0,-0.000000,-0.000244,-0.00011,-0.000085,0.000111
6,0,10000,5.498575,2.271809,0.101687,0.009148,460,25,"{'n_estimators': 460, 'max_depth': 25}",-0.0044,...,-0.0,-0.000275,-0.00022,-0.0,-0.0,-0.000000,-0.000244,-0.00011,-0.000085,0.000111
7,0,10000,7.249197,3.495222,0.152254,0.045997,620,25,"{'n_estimators': 620, 'max_depth': 25}",-0.0044,...,-0.0,-0.000275,-0.00022,-0.0,-0.0,-0.000000,-0.000244,-0.00011,-0.000085,0.000111
8,0,10000,8.292767,3.378441,0.156491,0.012501,700,30,"{'n_estimators': 700, 'max_depth': 30}",-0.0044,...,-0.0,-0.000275,-0.00022,-0.0,-0.0,-0.000000,-0.000244,-0.00011,-0.000085,0.000111
9,0,10000,3.072643,1.489956,0.069758,0.009977,250,70,"{'n_estimators': 250, 'max_depth': 70}",-0.0044,...,-0.0,-0.000275,-0.00022,-0.0,-0.0,-0.000000,-0.000244,-0.00011,-0.000085,0.000111


In [139]:
print(halving_search.best_params_)

{'n_estimators': 350, 'max_depth': 10}


In [140]:
rf = halving_search.best_estimator_

In [190]:
rf = joblib.load('./output/rf_02_autorreg.pkl').best_estimator_

In [191]:
importances = pd.DataFrame({
    'Feature': X_train.columns,
    'Importance': rf.feature_importances_
}).sort_values(by='Importance', ascending= False)
print(importances)

                                               Feature  Importance
129                                          default_1    0.356715
130                             prev_default_entidad_1    0.296873
131                             prev_default_general_1    0.213880
128                                         sin_arca_1    0.063665
133                                   buen_historial_1    0.021165
..                                                 ...         ...
49          Tasa de Política Monetaria (en % n.a.)_std    0.000005
47          Tasa de Política Monetaria (en % e.a.)_std    0.000005
14   BADLAR en pesos de bancos privados (en % n.a.)...    0.000004
109    Tasa de Política Monetaria (en % n.a.)_std_real    0.000003
107    Tasa de Política Monetaria (en % e.a.)_std_real    0.000002

[134 rows x 2 columns]


In [142]:
y_pred = eval(halving_search, X_test_cambio, Y_test_cambio)

[[  0  60]
 [434 159]]
El f1 es: 0.3916256157635468
El accuracy es: 0.2434915773353752
El MSE es: 0.7565084226646248
El AUC es: 0.13406408094435077


In [143]:
y_pred = eval(halving_search, X, Y)

[[876764    847]
 [  2779  58793]]
El f1 es: 0.9700854700854701
El accuracy es: 0.9961391975791726
El MSE es: 0.003860802420827464
El AUC es: 0.9769503640155511


## Modelo de Probabilidad Lineal

In [144]:
pipeline_lm = Pipeline([
    ('scaler', StandardScaler()),
    ('lm', LinearRegression(fit_intercept= True, n_jobs= -1))
                       ])
                    
pipeline_lm.fit(X_train, Y_train)

Pipeline(steps=[('scaler', StandardScaler()),
                ('lm', LinearRegression(n_jobs=-1))])

In [145]:
joblib.dump(pipeline_lm, './output/lm_02_autorreg.pkl')

['./output/lm_02_autorreg.pkl']

In [146]:
y_pred = eval(pipeline_lm, X_test, Y_test, linear= True)

[[176108     60]
 [   593  11076]]
El f1 es: 0.9713659285244464
El accuracy es: 0.9965235816159755
El MSE es: 0.0034764183840244466
El AUC es: 0.9744205041324834


In [147]:
lm = pipeline_lm.named_steps['lm']

In [197]:
df_coeficientes = pd.DataFrame({
    'variable': X_train.columns,
    'coeficiente': lm.coef_,
    'abs_coef': np.abs(lm.coef_)
}).sort_values(by = 'abs_coef', ascending= False)
print(df_coeficientes.iloc[:,:2])

                                              variable   coeficiente
129                                          default_1  1.988321e-01
130                             prev_default_entidad_1  4.593742e-02
128                                         sin_arca_1  2.040478e-03
132                                    sin_historial_1 -8.455806e-04
6                                       monto_relativo  2.607596e-04
..                                                 ...           ...
121  Tasas de interés por préstamos entre entidades...  1.408740e-06
31   En cuentas corrientes (neto de utilización FUC...  1.270815e-06
81   Depósitos de los bancos en cta. cte. en pesos ...  5.726163e-07
109    Tasa de Política Monetaria (en % n.a.)_std_real  3.596532e-07
60   Tasas de interés por préstamos entre entidades...  1.236592e-07

[134 rows x 2 columns]


In [149]:
y_pred = eval(pipeline_lm, X_test_cambio, Y_test_cambio, linear= True)

[[  0  60]
 [593   0]]
El f1 es: 0.0
El accuracy es: 0.0
El MSE es: 1.0
El AUC es: 0.0


In [150]:
y_pred = eval(pipeline_lm, X, Y, linear = True)

[[876843    768]
 [  3034  58538]]
El f1 es: 0.9685467992521385
El accuracy es: 0.995951800660787
El MSE es: 0.004048199339212912
El AUC es: 0.9749246261239876


## Regresión Logística

In [151]:
pipeline_logit = Pipeline([
    ('scaler', StandardScaler()),
	('logit', LogisticRegression(max_iter = 2000, fit_intercept = True, n_jobs = -1, solver = 'saga'))
])
pipeline_logit.fit(X_train, Y_train)

Pipeline(steps=[('scaler', StandardScaler()),
                ('logit',
                 LogisticRegression(max_iter=2000, n_jobs=-1, solver='saga'))])

In [152]:
joblib.dump(pipeline_logit, './output/logit_02_autorreg.pkl')

['./output/logit_02_autorreg.pkl']

In [153]:
y_pred = eval(pipeline_logit, X_test, Y_test)

[[176108     60]
 [   593  11076]]
El f1 es: 0.9713659285244464
El accuracy es: 0.9965235816159755
El MSE es: 0.0034764183840244466
El AUC es: 0.9744205041324834


In [154]:
logit = pipeline_logit.named_steps['logit']

In [201]:
df_coeficientes = pd.DataFrame({
    'variable': X_train.drop('default_1', axis = 1).columns,
    'coeficiente': logit.coef_[0],
    'abs_coef': np.abs(logit.coef_[0])
}).sort_values(by = 'abs_coef', ascending= False)
print(df_coeficientes.iloc[:, :2])

                                              variable  coeficiente
129                             prev_default_entidad_1     2.319560
132                                   buen_historial_1     0.731168
130                             prev_default_general_1     0.501823
128                                         sin_arca_1     0.332443
5                                    n_periodos_activo     0.285684
..                                                 ...          ...
45    TM20 en pesos de bancos privados (en % n.a.)_std     0.000504
76   Billetes y monedas en poder del público (en mi...     0.000404
28   Efectivo en entidades financieras (en millones...     0.000167
121  Tasas de interés por préstamos entre entidades...     0.000139
52   Tasa de interés de préstamos por adelantos en ...    -0.000128

[133 rows x 2 columns]


In [156]:
y_pred = eval(pipeline_logit, X_test_cambio, Y_test_cambio)

[[  0  60]
 [593   0]]
El f1 es: 0.0
El accuracy es: 0.0
El MSE es: 1.0
El AUC es: 0.0


In [157]:
y_pred = eval(pipeline_logit, X, Y)

[[876843    768]
 [  3034  58538]]
El f1 es: 0.9685467992521385
El accuracy es: 0.995951800660787
El MSE es: 0.004048199339212912
El AUC es: 0.9749246261239876


In [158]:
lasso = pipeline.named_steps['logreg']
best_c_lasso = lasso.C_[0]
rf = halving_search.best_estimator_
max_depth = rf.max_depth
n_estimators = rf.n_estimators

## VotingClassifier

In [159]:
logit_lasso = LogisticRegression(
    penalty="l1",
    C= best_c_lasso,  
    solver="saga",    
    random_state= 42
)

logit_no_reg = LogisticRegression(
    random_state= 42
)

rf = RandomForestClassifier(
    n_estimators = n_estimators,
    max_depth= max_depth,
    random_state= 42
)

estimadores = [
    ("logit_lasso", logit_lasso),
    ("logit_no_reg", logit_no_reg),
    ("random_forest", rf)
]

#### Soft

In [160]:
pipeline_voting = Pipeline([
	('scaler', StandardScaler()),
	('voting', VotingClassifier(
     estimators=estimadores,
     voting="soft",
     n_jobs = -1  
))])

In [161]:
tscv = TimeSeriesSplit(n_splits = 5)

param_distributions = {
    'voting__weights': [np.random.dirichlet(np.ones(len(estimadores))) for _ in range(80)]
}

halving_search_vote = HalvingRandomSearchCV(
    estimator=pipeline_voting,
    param_distributions=param_distributions,
    factor=3,  
    min_resources=10000,  
    resource="n_samples",
    scoring='neg_mean_squared_error',
    n_jobs=-1,
    cv=tscv,
    random_state=42,
    verbose=4
)

In [162]:
halving_search_vote.fit(X_train, Y_train)

n_iterations: 4
n_required_iterations: 4
n_possible_iterations: 4
min_resources_: 10000
max_resources_: 751346
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 75
n_resources: 10000
Fitting 5 folds for each of 75 candidates, totalling 375 fits
----------
iter: 1
n_candidates: 25
n_resources: 30000
Fitting 5 folds for each of 25 candidates, totalling 125 fits
----------
iter: 2
n_candidates: 9
n_resources: 90000
Fitting 5 folds for each of 9 candidates, totalling 45 fits
----------
iter: 3
n_candidates: 3
n_resources: 270000
Fitting 5 folds for each of 3 candidates, totalling 15 fits


HalvingRandomSearchCV(cv=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=5, test_size=None),
                      estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                                ('voting',
                                                 VotingClassifier(estimators=[('logit_lasso',
                                                                               LogisticRegression(C=0.00026366508987303583,
                                                                                                  penalty='l1',
                                                                                                  random_state=42,
                                                                                                  solver='saga')),
                                                                              ('logit_no_reg',
                                                                               LogisticRegression(random_state=42)),
                                                                              ('rand...
                                                               array([0.40228405, 0.37859221, 0.21912374]),
                                                               array([0.65460272, 0.03233446, 0.31306283]),
                                                               array([0.14521159, 0.39875614, 0.45603227]),
                                                               array([0.01833412, 0.79175495, 0.18991092]),
                                                               array([0.06173908, 0.49367621, 0.44458471]),
                                                               array([0.5711429 , 0.16698898, 0.26186812]),
                                                               array([0.41520607, 0.03449788, 0.55029605]), ...]},
                      random_state=42, scoring='neg_mean_squared_error',
                      verbose=4)

In [163]:
joblib.dump(halving_search_vote, './output/voting_02_autorreg.pkl')

['./output/voting_02_autorreg.pkl']

In [164]:
y_pred = eval(halving_search_vote, X_test, Y_test)

[[176108     60]
 [   593  11076]]
El f1 es: 0.9713659285244464
El accuracy es: 0.9965235816159755
El MSE es: 0.0034764183840244466
El AUC es: 0.9744205041324834


In [165]:
results_voting = pd.DataFrame(halving_search_vote.cv_results_)
results_voting.to_csv('./output/results_voting_02_autorreg.csv')

In [166]:
voting = halving_search_vote.best_estimator_

In [167]:
print(halving_search_vote.best_params_)

{'voting__weights': array([0.40228405, 0.37859221, 0.21912374])}


In [168]:
y_pred = eval(halving_search_vote, X_test_cambio, Y_test_cambio)

[[  0  60]
 [593   0]]
El f1 es: 0.0
El accuracy es: 0.0
El MSE es: 1.0
El AUC es: 0.0


In [169]:
y_pred = eval(halving_search_vote, X, Y)

[[876843    768]
 [  3034  58538]]
El f1 es: 0.9685467992521385
El accuracy es: 0.995951800660787
El MSE es: 0.004048199339212912
El AUC es: 0.9749246261239876


# Sin Autorregresión

## Evaluación sobre el conjunto de testeo

| Modelo              | TN     | FP  | FN  | TP     | F1        | Accuracy      | MSE              | AUC          |
|---------------------|--------|-----|-----|--------|-----------|--------------|------------------|--------------|
| LASSO              | 175971 | 197 | 224 | 11445  | 0.98194   | 0.99776      | 0.00224          | 0.98984      |
| Random Forest      | 175971 | 197 | 217 | 11452  | 0.98225   | 0.99780      | 0.00220          | 0.99014      |
| Logit              | 175971 | 197 | 220 | 11449  | 0.98211   | 0.99778      | 0.00222          | 0.99001      |
| Linear Regression  | 175971 | 197 | 217 | 11452  | 0.98225   | 0.99780      | 0.00220          | 0.99014      |
| Voting Classifier  | 175971 | 197 | 217 | 11452  | 0.98225   | 0.99780      | 0.00220          | 0.99014      |


## Evaluado sobre las observaciones cuyo $default_t \neq default_{t+1}$

| Modelo             | TN  | FP | FN  | TP | F1        | Accuracy      | MSE              | AUC          |
|--------------------|----|----|----|----|-----------|--------------|------------------|--------------|
| LASSO             |  0 | 60 | 217 | 376 | 0.73081   | 0.57580      | 0.42420          | 0.31703      |
| Random Forest     |  0 | 60 | 217 | 376 | 0.73081   | 0.57580      | 0.42420          | 0.31703      |
| Logit             |  0 | 60 | 217 | 376 | 0.73081   | 0.57580      | 0.42420          | 0.31703      |
| Linear Regression |  0 | 60 | 217 | 376 | 0.73081   | 0.57580      | 0.42420          | 0.31703      |
| Voting Classifier |  0 | 60 | 217 | 376 | 0.73081   | 0.57580      | 0.42420          | 0.31703      |

## Evaluado sobre toda la muestra (entrenamiento + testeo)

| Modelo             | TN    | FP    | FN  | TP    | F1        | Accuracy      | MSE              | AUC          |
|--------------------|-------|-------|-----|-------|-----------|--------------|------------------|--------------|
| LASSO             | 866421 | 11190 | 506 | 61066 | 0.91260   | 0.98755      | 0.01245          | 0.98952      |
| Random Forest     | 867755 | 9856  | 786 | 60786 | 0.91951   | 0.98867      | 0.01133          | 0.98800      |
| Logit             | 866428 | 11183 | 504 | 61068 | 0.91267   | 0.98756      | 0.01244          | 0.98954      |
| Linear Regression | 866081 | 11530 | 455 | 61117 | 0.91071   | 0.98724      | 0.01276          | 0.98974      |
| Voting Classifier | 866452 | 11159 | 496 | 61076 | 0.91290   | 0.98759      | 0.01241          | 0.98961      |


# Con Autorregresión

## Evaluación sobre el conjunto de testeo

| Modelo             | TN    | FP  | FN  | TP    | F1        | Accuracy      | MSE              | AUC          |
|--------------------|-------|-----|-----|-------|-----------|--------------|------------------|--------------|
| LASSO             | 176108 | 60  | 593 | 11076 | 0.97137   | 0.99652      | 0.00348          | 0.97442      |
| Random Forest     | 176032 | 136 | 434 | 11235 | 0.97526   | 0.99697      | 0.00303          | 0.98102      |
| Logit             | 176108 | 60  | 593 | 11076 | 0.97137   | 0.99652      | 0.00348          | 0.97442      |
| Linear Regression | 176108 | 60  | 593 | 11076 | 0.97137   | 0.99652      | 0.00348          | 0.97442      |
| Voting Classifier | 176108 | 60  | 593 | 11076 | 0.97137   | 0.99652      | 0.00348          | 0.97442      |



## Evaluado sobre las observaciones cuyo $default_t \neq default_{t+1}$

| Modelo            | TN  | FP  | FN  | TP  | F1 Score | Accuracy | MSE  | AUC  |
|------------------|----|----|----|----|----------|----------|------|------|
| LASSO           |  0 | 60 | 593 |  0 | 0.0000   | 0.0000   | 1.0000 | 0.0000 |
| Random Forest   | 0 | 60 | 434 |  159 | 0.3916   | 0.2435   | 0.7565 | 0.1341 |
| Logit           |  0 | 60 | 593 |  0 | 0.0000   | 0.0000   | 1.0000 | 0.0000 |
| Linear Regression | 0 | 60 | 593 |  0 | 0.0000   | 0.0000   | 1.0000 | 0.0000 |
| Voting Classifier | 0 | 60 | 593 |  0 | 0.0000   | 0.0000   | 1.0000 | 0.0000 |


## Evaluado sobre toda la muestra (entrenamiento + testeo)

| Modelo            | TP     | FP   | FN   | TN     | F1 Score | Accuracy | MSE   | AUC   |
|------------------|--------|------|------|--------|----------|----------|-------|-------|
| LASSO           | 876843  | 768  | 3034 | 58538 | 0.9685   | 0.99595  | 0.0040 | 0.9749 |
| Random Forest   |  876764 | 847  | 2779 | 58793 | 0.9701   | 0.99614  | 0.0039 | 0.9770 |
| Logit           | 876843  | 768  | 3034 | 58538 | 0.9685   | 0.99595  | 0.0040 | 0.9749 |
| Linear Regression | 876843  | 768  | 3034 | 58538 | 0.9685   | 0.99595  | 0.0040 | 0.9749 |
| Voting Classifier |  876843 | 768  | 3034 | 58538 | 0.9685   | 0.99595  | 0.0040 | 0.9749 |



In [170]:
print('Cantidad de Créditos que pasaron de estar defaulteados a estar al día:')
print(len(Y_test_cambio.loc[Y_test_cambio == 0]))
print('Cantidad de créditos que pasaron de estar al día a estar defaulteados:')
print(len(Y_test_cambio.loc[Y_test_cambio == 1]))

Cantidad de Créditos que pasaron de estar defaulteados a estar al día:
60
Cantidad de créditos que pasaron de estar al día a estar defaulteados:
593


In [171]:
print('Proporción de créditos al día:')
print(len(Y_test.loc[Y_test == 0])/len(Y_test))

Proporción de créditos al día:
0.937876989091606


Un predictor trivial que siempre prediga que el crédito no va a estar defaulteado le pegaría el 93.78% de las veces

In [172]:
print('LASSO:')
y_pred = eval(pipeline, X_test, Y_test)
print('Random Forest:')
y_pred = eval(halving_search, X_test, Y_test)
print('Logit:')
y_pred = eval(pipeline_logit, X_test, Y_test)
print('Linear Regression:')
y_pred = eval(pipeline_lm, X_test, Y_test, linear = True)
print('Voting Classifier:')
y_pred = eval(halving_search_vote, X_test, Y_test)

LASSO:
[[176108     60]
 [   593  11076]]
El f1 es: 0.9713659285244464
El accuracy es: 0.9965235816159755
El MSE es: 0.0034764183840244466
El AUC es: 0.9744205041324834
Random Forest:
[[176032    136]
 [   434  11235]]
El f1 es: 0.9752604166666666
El accuracy es: 0.9969654540905146
El MSE es: 0.0030345459094853517
El AUC es: 0.9810177240697358
Logit:
[[176108     60]
 [   593  11076]]
El f1 es: 0.9713659285244464
El accuracy es: 0.9965235816159755
El MSE es: 0.0034764183840244466
El AUC es: 0.9744205041324834
Linear Regression:
[[176108     60]
 [   593  11076]]
El f1 es: 0.9713659285244464
El accuracy es: 0.9965235816159755
El MSE es: 0.0034764183840244466
El AUC es: 0.9744205041324834
Voting Classifier:
[[176108     60]
 [   593  11076]]
El f1 es: 0.9713659285244464
El accuracy es: 0.9965235816159755
El MSE es: 0.0034764183840244466
El AUC es: 0.9744205041324834


In [173]:
print('LASSO:')
y_pred = eval(pipeline, X_test_cambio, Y_test_cambio)
print('Random Forest:')
y_pred = eval(halving_search, X_test_cambio, Y_test_cambio)
print('Logit:')
y_pred = eval(pipeline_logit, X_test_cambio, Y_test_cambio)
print('Linear Regression:')
y_pred = eval(pipeline_lm, X_test_cambio, Y_test_cambio, linear = True)
print('Voting Classifier:')
y_pred = eval(halving_search_vote, X_test_cambio, Y_test_cambio)

LASSO:
[[  0  60]
 [593   0]]
El f1 es: 0.0
El accuracy es: 0.0
El MSE es: 1.0
El AUC es: 0.0
Random Forest:
[[  0  60]
 [434 159]]
El f1 es: 0.3916256157635468
El accuracy es: 0.2434915773353752
El MSE es: 0.7565084226646248
El AUC es: 0.13406408094435077
Logit:
[[  0  60]
 [593   0]]
El f1 es: 0.0
El accuracy es: 0.0
El MSE es: 1.0
El AUC es: 0.0
Linear Regression:
[[  0  60]
 [593   0]]
El f1 es: 0.0
El accuracy es: 0.0
El MSE es: 1.0
El AUC es: 0.0
Voting Classifier:
[[  0  60]
 [593   0]]
El f1 es: 0.0
El accuracy es: 0.0
El MSE es: 1.0
El AUC es: 0.0


In [174]:
print('LASSO:')
y_pred = eval(pipeline, X, Y)
print('Random Forest:')
y_pred = eval(halving_search, X, Y)
print('Logit:')
y_pred = eval(pipeline_logit, X, Y)
print('Linear Regression:')
y_pred = eval(pipeline_lm, X, Y, linear = True)
print('Voting Classifier:')
y_pred = eval(halving_search_vote, X, Y)

LASSO:
[[876843    768]
 [  3034  58538]]
El f1 es: 0.9685467992521385
El accuracy es: 0.995951800660787
El MSE es: 0.004048199339212912
El AUC es: 0.9749246261239876
Random Forest:
[[876764    847]
 [  2779  58793]]
El f1 es: 0.9700854700854701
El accuracy es: 0.9961391975791726
El MSE es: 0.003860802420827464
El AUC es: 0.9769503640155511
Logit:
[[876843    768]
 [  3034  58538]]
El f1 es: 0.9685467992521385
El accuracy es: 0.995951800660787
El MSE es: 0.004048199339212912
El AUC es: 0.9749246261239876
Linear Regression:
[[876843    768]
 [  3034  58538]]
El f1 es: 0.9685467992521385
El accuracy es: 0.995951800660787
El MSE es: 0.004048199339212912
El AUC es: 0.9749246261239876
Voting Classifier:
[[876843    768]
 [  3034  58538]]
El f1 es: 0.9685467992521385
El accuracy es: 0.995951800660787
El MSE es: 0.004048199339212912
El AUC es: 0.9749246261239876
